## Inverse Kinematics: FABRIK(Foward and Backward Reaching Kinematics)

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Arm:
    def __init__(self, _ax, _ay, _length, _angle):
        self.ax = _ax
        self.ay = _ay
        self.length = _length
        self.angle = _angle
        self.bx = self.ax + self.length * np.cos(self.angle)
        self.by = self.ay + self.length * np.sin(self.angle)

    def backward(self, tx, ty):
        theta = np.arctan2(ty - self.ay, tx - self.ax)
        self.ax = tx - self.length * np.cos(theta)
        self.ay = ty - self.length * np.sin(theta)
        self.bx = tx
        self.by = ty
        
    def forward(self, baseX, baseY):
        theta = np.arctan2(self.by - baseY, self.bx - baseX)
        self.ax = baseX
        self.ay = baseY
        self.bx = baseX + self.length * np.cos(theta)
        self.by = baseY + self.length * np.sin(theta)

In [3]:
N = 4  # the number of link
OX = 0 # offset x: Base coodinates
OY = 0 # offset y: Base coodinates

fab = []
for i in range(N):
    linkLength = 1 - i * 0.1
    initTheta = np.pi / 2   # vertical position
    if i == 0:
        fab.append(Arm(OX, OY, linkLength, initTheta))
    else:
        fab.append(Arm(fab[i-1].bx, fab[i-1].by, linkLength, initTheta))
        
fabX = []
fabY = []
for i in range(N):
    fabX.append([fab[i].ax, fab[i].bx])
    fabY.append([fab[i].ay, fab[i].by])
        
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.axis([-1,N,-1,N])
ax.grid()
ax.plot(fabX, fabY, color='orange')
ax.scatter(fabX, fabY, color='orange')

<IPython.core.display.Javascript object>

In [4]:
# FABRIK: Backward + Forward

def FABRIK(arm, _tx, _ty):
    tx = _tx
    ty = _ty
    for i in reversed(range(N)):
        arm[i].backward(tx, ty)
        tx = arm[i].ax
        ty = arm[i].ay
    
    baseX = OX
    baseY = OY
    for i in range(N):
        arm[i].forward(baseX, baseY);
        baseX = arm[i].bx
        baseY = arm[i].by
        
    PX = []
    PY = []
    for i in range(N):
        PX.append(arm[i].ax)
        PY.append(arm[i].ay)  
    PX.append(arm[N-1].bx)
    PY.append(arm[N-1].by)
    
    return PX, PY

In [5]:
def motion(event):
    mx = event.xdata
    my = event.ydata
    Mouse.set_data(mx, my)
    
    FX, FY = FABRIK(fab, mx, my)
    FLine.set_data(FX, FY)
    FDot.set_data(FX, FY)
    
    plt.draw()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.axis([-1,N,-1,N])
ax.grid()

FLine, = ax.plot([],[], linestyle='-', color='orange', alpha=1, label='FABRIK')
FDot, = ax.plot([],[], marker='o', color='orange')
Mouse, = ax.plot([],[], marker='x', ms=20, color='red')

plt.connect('motion_notify_event', motion)
plt.legend(loc='lower right')
plt.show()

<IPython.core.display.Javascript object>